# Verificación de Dataset en Google Colab

Este notebook te permite:
1. Descargar el dataset en Colab
2. Calcular estadísticas y hash para comparar con tu versión local
3. Guardar el dataset en Google Drive para inspección manual

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Instalar paquetes necesarios
!pip install rpy2

In [ ]:
%%writefile /content/verify_dataset.R

# Cargar librerías necesarias
library(data.table)
library(digest)  # Para calcular hash

# Descargar dataset
cat("\n=== DESCARGANDO DATASET ===\n")
dataset_url <- "https://storage.googleapis.com/open-courses/austral2025-af91/gerencial_competencia_2025.csv.gz"
dataset_file <- "/content/gerencial_competencia_2025.csv.gz"

download.file(dataset_url, dataset_file, mode = "wb")
cat("Dataset descargado exitosamente\n")

# Verificar tamaño del archivo
file_size <- file.info(dataset_file)$size
cat(sprintf("\nTamaño del archivo: %.2f MB\n", file_size / (1024^2)))

# Calcular MD5 hash del archivo comprimido
cat("\n=== HASH DEL ARCHIVO COMPRIMIDO ===\n")
md5_gz <- digest(dataset_file, algo = "md5", file = TRUE)
cat(sprintf("MD5 (.csv.gz): %s\n", md5_gz))

# Cargar dataset
cat("\n=== CARGANDO DATASET ===\n")
dataset <- fread(dataset_file)
cat("Dataset cargado exitosamente\n")

# Estadísticas básicas
cat("\n=== ESTADÍSTICAS BÁSICAS ===\n")
cat(sprintf("Número de filas: %d\n", nrow(dataset)))
cat(sprintf("Número de columnas: %d\n", ncol(dataset)))
cat(sprintf("Memoria utilizada: %.2f MB\n", object.size(dataset) / (1024^2)))

# Información de foto_mes
cat("\n=== DISTRIBUCIÓN DE FOTO_MES ===\n")
foto_mes_dist <- dataset[, .N, by = foto_mes][order(foto_mes)]
print(foto_mes_dist)

# Información de clase_ternaria
cat("\n=== DISTRIBUCIÓN DE CLASE_TERNARIA ===\n")
clase_dist <- dataset[, .N, by = clase_ternaria]
print(clase_dist)

# Primeras y últimas columnas
cat("\n=== NOMBRES DE COLUMNAS ===\n")
cat("Primeras 10 columnas:\n")
print(head(names(dataset), 10))
cat("\nÚltimas 10 columnas:\n")
print(tail(names(dataset), 10))

# Estadísticas de algunas columnas clave
cat("\n=== ESTADÍSTICAS DE COLUMNAS CLAVE ===\n")
if("numero_de_cliente" %in% names(dataset)) {
  cat(sprintf("Clientes únicos: %d\n", uniqueN(dataset$numero_de_cliente)))
}

if("mcuentas_saldo" %in% names(dataset)) {
  cat(sprintf("mcuentas_saldo - Media: %.2f, SD: %.2f, NAs: %d\n", 
              mean(dataset$mcuentas_saldo, na.rm = TRUE),
              sd(dataset$mcuentas_saldo, na.rm = TRUE),
              sum(is.na(dataset$mcuentas_saldo))))
}

if("mtarjeta_visa_consumo" %in% names(dataset)) {
  cat(sprintf("mtarjeta_visa_consumo - Media: %.2f, SD: %.2f, NAs: %d\n", 
              mean(dataset$mtarjeta_visa_consumo, na.rm = TRUE),
              sd(dataset$mtarjeta_visa_consumo, na.rm = TRUE),
              sum(is.na(dataset$mtarjeta_visa_consumo))))
}

# Calcular hash de los datos (primeras 1000 filas para velocidad)
cat("\n=== HASH DE DATOS (primeras 1000 filas) ===\n")
data_sample <- head(dataset, 1000)
data_hash <- digest(data_sample, algo = "md5")
cat(sprintf("MD5 (primeras 1000 filas): %s\n", data_hash))

# Hash de toda la columna clase_ternaria
cat("\n=== HASH DE COLUMNA CLASE_TERNARIA ===\n")
clase_hash <- digest(dataset$clase_ternaria, algo = "md5")
cat(sprintf("MD5 (clase_ternaria completa): %s\n", clase_hash))

# Guardar resumen en archivo
cat("\n=== GUARDANDO RESUMEN ===\n")
summary_file <- "/content/drive/MyDrive/dataset_verification_colab.txt"
sink(summary_file)
cat("VERIFICACIÓN DE DATASET - GOOGLE COLAB\n")
cat("========================================\n\n")
cat(sprintf("Fecha: %s\n\n", Sys.time()))
cat(sprintf("Tamaño archivo: %.2f MB\n", file_size / (1024^2)))
cat(sprintf("MD5 archivo .gz: %s\n\n", md5_gz))
cat(sprintf("Filas: %d\n", nrow(dataset)))
cat(sprintf("Columnas: %d\n\n", ncol(dataset)))
cat("Distribución foto_mes:\n")
print(foto_mes_dist)
cat("\nDistribución clase_ternaria:\n")
print(clase_dist)
cat(sprintf("\nMD5 (primeras 1000 filas): %s\n", data_hash))
cat(sprintf("MD5 (clase_ternaria): %s\n", clase_hash))
sink()

cat(sprintf("Resumen guardado en: %s\n", summary_file))

# Opcionalmente, guardar el dataset en Drive (descomenta si quieres una copia)
# cat("\n=== GUARDANDO DATASET EN DRIVE ===\n")
# dataset_copy <- "/content/drive/MyDrive/gerencial_competencia_2025_from_colab.csv.gz"
# fwrite(dataset, dataset_copy)
# cat(sprintf("Dataset guardado en: %s\n", dataset_copy))

cat("\n=== VERIFICACIÓN COMPLETA ===\n")

In [ ]:
# Ejecutar el script R
%load_ext rpy2.ipython

In [ ]:
%%bash
Rscript /content/verify_dataset.R

## Instrucciones para comparar con dataset local

1. **En tu máquina local**, corre el siguiente script R para obtener las mismas estadísticas:

```r
library(data.table)
library(digest)

# Ajusta la ruta a tu dataset local
dataset_file <- "C:/Users/User/Documents/labo2025v/datasets/gerencial_competencia_2025.csv.gz"

# Calcular hash del archivo
md5_gz <- digest(dataset_file, algo = "md5", file = TRUE)
cat(sprintf("MD5 (.csv.gz): %s\n", md5_gz))

# Cargar y analizar
dataset <- fread(dataset_file)
cat(sprintf("Filas: %d, Columnas: %d\n", nrow(dataset), ncol(dataset)))

# Hash de primeras 1000 filas
data_hash <- digest(head(dataset, 1000), algo = "md5")
cat(sprintf("MD5 (primeras 1000 filas): %s\n", data_hash))

# Hash de clase_ternaria
clase_hash <- digest(dataset$clase_ternaria, algo = "md5")
cat(sprintf("MD5 (clase_ternaria): %s\n", clase_hash))
```

2. **Compara** los hashes:
   - Si los MD5 del archivo .gz son **iguales** → los archivos son idénticos
   - Si los MD5 de las primeras 1000 filas son **iguales** → los datos descomprimidos son iguales
   - Si el MD5 de clase_ternaria es **igual** → la variable target es la misma

3. El archivo `dataset_verification_colab.txt` estará en tu Google Drive con todos los detalles.